In [3]:
from IPython.display import display

# knowns about the data 

the csvs given are  uniform, so before preparing for any data analysis I'm going to combine them into one main file. Then performing some analysis on the combined data to get a complete picture of the data

In [5]:
import glob
import pandas as pd

files = glob.glob("data/JC-2016*.csv")


bike_df = []

for fname in files:
    data  = pd.read_csv(fname)
    bike_df.append(data)

combined_bike_df = pd.concat(bike_df)

print('data files combined...completed')

data files combined...completed


## basic stats overview

In [6]:
combined_bike_df.dtypes

Trip Duration                int64
Start Time                  object
Stop Time                   object
Start Station ID             int64
Start Station Name          object
Start Station Latitude     float64
Start Station Longitude    float64
End Station ID               int64
End Station Name            object
End Station Latitude       float64
End Station Longitude      float64
Bike ID                      int64
User Type                   object
Birth Year                 float64
Gender                       int64
dtype: object

In [7]:
combined_bike_df.nunique()

Trip Duration                6024
Start Time                 244407
Stop Time                  244137
Start Station ID               51
Start Station Name             51
Start Station Latitude         51
Start Station Longitude        51
End Station ID                102
End Station Name              102
End Station Latitude          102
End Station Longitude         102
Bike ID                       566
User Type                       2
Birth Year                     64
Gender                          3
dtype: int64

## Finding where the data is missing

In [10]:
# counts the number of missing values in each column 
combined_bike_df.isna().sum() 

Trip Duration                  0
Start Time                     0
Stop Time                      0
Start Station ID               0
Start Station Name             0
Start Station Latitude         0
Start Station Longitude        0
End Station ID                 0
End Station Name               0
End Station Latitude           0
End Station Longitude          0
Bike ID                        0
User Type                    380
Birth Year                 18999
Gender                         0
dtype: int64

## Finally viewing the data at high level

At this point, there is two columns of concern and I think I just want to check out the data from a tabular view to get a better feel.

In [8]:
display(combined_bike_df.head(10))

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,362,2016-01-01 00:02:52,2016-01-01 00:08:54,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24647,Subscriber,1964.0,2
1,200,2016-01-01 00:18:22,2016-01-01 00:21:42,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24605,Subscriber,1962.0,1
2,202,2016-01-01 00:18:25,2016-01-01 00:21:47,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24689,Subscriber,1962.0,2
3,248,2016-01-01 00:23:13,2016-01-01 00:27:21,3209,Brunswick St,40.724176,-74.050656,3203,Hamilton Park,40.727596,-74.044247,24693,Subscriber,1984.0,1
4,903,2016-01-01 01:03:20,2016-01-01 01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24573,Customer,NaN,0
5,883,2016-01-01 01:03:28,2016-01-01 01:18:11,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24442,Customer,NaN,0
6,445,2016-01-01 01:07:45,2016-01-01 01:15:11,3186,Grove St PATH,40.719586,-74.043117,3203,Hamilton Park,40.727596,-74.044247,24510,Subscriber,1988.0,2
7,192,2016-01-01 01:18:51,2016-01-01 01:22:03,3211,Newark Ave,40.721525,-74.046305,3203,Hamilton Park,40.727596,-74.044247,24625,Subscriber,1980.0,1
8,409,2016-01-01 01:23:44,2016-01-01 01:30:34,3187,Warren St,40.721124,-74.038051,3214,Essex Light Rail,40.712774,-74.036486,24429,Subscriber,1990.0,1
9,285,2016-01-01 01:25:12,2016-01-01 01:29:57,3187,Warren St,40.721124,-74.038051,3214,Essex Light Rail,40.712774,-74.036486,24407,Subscriber,1988.0,2


## final assessments of the data

the columns "birth year" and "User type" have a large amount of null data within them.  I would want to account for that or possibly create a view seperate from the main table within the database.  

Before ingesting this into the database the main thing(S) I'd like to tackle is: breaking up the date and time, I feel that more grainular data is better for analzying down the road, if needed.  The the trip duration is listed in seconds from the data dictionary

------------------------------------------------------------------------------

# Preparing the data

This data will go into a Postgres database and from there I'll make some views to get the data into more digestable forms and anaylyze from there.

However, the data needs to be transformed a bit before getting it into the database.  I'd like to add some column(s) like units for time on ride duration, split up the "Start Time" and "Stop Time" where the data and the time is in different columns

In [9]:
display(combined_bike_df['Start Time'].head(10))

0    2016-01-01 00:02:52
1    2016-01-01 00:18:22
2    2016-01-01 00:18:25
3    2016-01-01 00:23:13
4    2016-01-01 01:03:20
5    2016-01-01 01:03:28
6    2016-01-01 01:07:45
7    2016-01-01 01:18:51
8    2016-01-01 01:23:44
9    2016-01-01 01:25:12
Name: Start Time, dtype: object

In [10]:
display(combined_bike_df['Stop Time'].head(10))

0    2016-01-01 00:08:54
1    2016-01-01 00:21:42
2    2016-01-01 00:21:47
3    2016-01-01 00:27:21
4    2016-01-01 01:18:24
5    2016-01-01 01:18:11
6    2016-01-01 01:15:11
7    2016-01-01 01:22:03
8    2016-01-01 01:30:34
9    2016-01-01 01:29:57
Name: Stop Time, dtype: object

In [11]:
start_date_split = combined_bike_df['Start Time'].str.split(' ')
combined_bike_df['Start Date'] = start_date_split.str.get(0)
combined_bike_df['Start Time'] = start_date_split.str.get(1)

start_date_split = combined_bike_df['Stop Time'].str.split(' ')
combined_bike_df['Stop Date'] = start_date_split.str.get(0)
combined_bike_df['Stop Time'] = start_date_split.str.get(1)

In [13]:
print(combined_bike_df['Start Date'].head(5))
print(combined_bike_df['Start Time'].head(5))

print(combined_bike_df['Stop Date'].head(5))
print(combined_bike_df['Stop Time'].head(5))

0    2016-01-01
1    2016-01-01
2    2016-01-01
3    2016-01-01
4    2016-01-01
Name: Start Date, dtype: object
0    00:02:52
1    00:18:22
2    00:18:25
3    00:23:13
4    01:03:20
Name: Start Time, dtype: object
0    2016-01-01
1    2016-01-01
2    2016-01-01
3    2016-01-01
4    2016-01-01
Name: Stop Date, dtype: object
0    00:08:54
1    00:21:42
2    00:21:47
3    00:27:21
4    01:18:24
Name: Stop Time, dtype: object


In [14]:
display(combined_bike_df.head(10))

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,Start Date,Stop Date
0,362,00:02:52,00:08:54,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24647,Subscriber,1964.0,2,2016-01-01,2016-01-01
1,200,00:18:22,00:21:42,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24605,Subscriber,1962.0,1,2016-01-01,2016-01-01
2,202,00:18:25,00:21:47,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24689,Subscriber,1962.0,2,2016-01-01,2016-01-01
3,248,00:23:13,00:27:21,3209,Brunswick St,40.724176,-74.050656,3203,Hamilton Park,40.727596,-74.044247,24693,Subscriber,1984.0,1,2016-01-01,2016-01-01
4,903,01:03:20,01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24573,Customer,NaN,0,2016-01-01,2016-01-01
5,883,01:03:28,01:18:11,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24442,Customer,NaN,0,2016-01-01,2016-01-01
6,445,01:07:45,01:15:11,3186,Grove St PATH,40.719586,-74.043117,3203,Hamilton Park,40.727596,-74.044247,24510,Subscriber,1988.0,2,2016-01-01,2016-01-01
7,192,01:18:51,01:22:03,3211,Newark Ave,40.721525,-74.046305,3203,Hamilton Park,40.727596,-74.044247,24625,Subscriber,1980.0,1,2016-01-01,2016-01-01
8,409,01:23:44,01:30:34,3187,Warren St,40.721124,-74.038051,3214,Essex Light Rail,40.712774,-74.036486,24429,Subscriber,1990.0,1,2016-01-01,2016-01-01
9,285,01:25:12,01:29:57,3187,Warren St,40.721124,-74.038051,3214,Essex Light Rail,40.712774,-74.036486,24407,Subscriber,1988.0,2,2016-01-01,2016-01-01


In [20]:
combined_bike_df['Fixed Trip Duration'] = combined_bike_df['Trip Duration'].apply(lambda x: '{:02d} minutes {:02d} seconds'.format(x // 60, x % 60))


In [21]:
display(combined_bike_df.head(10))

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,Start Date,Stop Date,Fixed Trip Duration
0,362,00:02:52,00:08:54,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24647,Subscriber,1964.0,2,2016-01-01,2016-01-01,06 minutes 02 seconds
1,200,00:18:22,00:21:42,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24605,Subscriber,1962.0,1,2016-01-01,2016-01-01,03 minutes 20 seconds
2,202,00:18:25,00:21:47,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24689,Subscriber,1962.0,2,2016-01-01,2016-01-01,03 minutes 22 seconds
3,248,00:23:13,00:27:21,3209,Brunswick St,40.724176,-74.050656,3203,Hamilton Park,40.727596,-74.044247,24693,Subscriber,1984.0,1,2016-01-01,2016-01-01,04 minutes 08 seconds
4,903,01:03:20,01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24573,Customer,NaN,0,2016-01-01,2016-01-01,15 minutes 03 seconds
5,883,01:03:28,01:18:11,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24442,Customer,NaN,0,2016-01-01,2016-01-01,14 minutes 43 seconds
6,445,01:07:45,01:15:11,3186,Grove St PATH,40.719586,-74.043117,3203,Hamilton Park,40.727596,-74.044247,24510,Subscriber,1988.0,2,2016-01-01,2016-01-01,07 minutes 25 seconds
7,192,01:18:51,01:22:03,3211,Newark Ave,40.721525,-74.046305,3203,Hamilton Park,40.727596,-74.044247,24625,Subscriber,1980.0,1,2016-01-01,2016-01-01,03 minutes 12 seconds
8,409,01:23:44,01:30:34,3187,Warren St,40.721124,-74.038051,3214,Essex Light Rail,40.712774,-74.036486,24429,Subscriber,1990.0,1,2016-01-01,2016-01-01,06 minutes 49 seconds
9,285,01:25:12,01:29:57,3187,Warren St,40.721124,-74.038051,3214,Essex Light Rail,40.712774,-74.036486,24407,Subscriber,1988.0,2,2016-01-01,2016-01-01,04 minutes 45 seconds


In [23]:
combined_bike_df.to_csv('fixed_bike_data.csv', index=False)